In [1]:
from core.runner import Runner
import tensorflow as tf

grid_length = 8

runner = Runner(
    grid_length=grid_length,
    main_layer_hidden_nodes=[80, 40, 20],
    branch1_hidden_nodes=[10],
    branch2_hidden_nodes=[10],
    exploration_rate=0.5,
    learning_rate=0.005,
    environment_mode="spinfoam_vertex"
)

2022-12-12 18:22:54.047588: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [ ]:
import time

batch_size = tf.constant(500)
n_iterations = tf.constant(int(5e4))
check_loss_every_n_iterations = tf.constant(int(1e3))

start = time.time()
ave_losses = runner.train_agent(batch_size, n_iterations, check_loss_every_n_iterations)
time_elapsed = time.time() - start
print(f"Time Elapsed: {time_elapsed} s")

Iteration: 0 Average Loss: 29.4460258
Iteration: 1000 Average Loss: 2.94355965
Iteration: 2000 Average Loss: 2.65810752
Iteration: 3000 Average Loss: 2.38932133
Iteration: 4000 Average Loss: 2.12302446
Iteration: 5000 Average Loss: 2.5067749
Iteration: 6000 Average Loss: 1.58465111
Iteration: 7000 Average Loss: 1.44875538
Iteration: 8000 Average Loss: 1.57229376
Iteration: 9000 Average Loss: 1.60856199
Iteration: 10000 Average Loss: 1.59234273
Iteration: 11000 Average Loss: 1.56522393
Iteration: 12000 Average Loss: 1.70676792
Iteration: 13000 Average Loss: 1.43888962
Iteration: 14000 Average Loss: 1.27018249
Iteration: 15000 Average Loss: 1.40723062
Iteration: 16000 Average Loss: 1.28698468
Iteration: 17000 Average Loss: 1.50784612
Iteration: 18000 Average Loss: 1.35830343
Iteration: 19000 Average Loss: 1.4558382
Iteration: 20000 Average Loss: 1.34285867
Iteration: 21000 Average Loss: 1.37678421
Iteration: 22000 Average Loss: 1.67767429
